# Generating input for FMTOMO from an ObsPy catalogue

We provide here a small set of utilities that produce the required inputs for [FMTomo](http://rses.anu.edu.au/~nick/fmtomo.html) from an [ObsPy Catalog](https://docs.obspy.org/packages/autogen/obspy.core.event.Catalog.html) - namely:

1. A collection of files containing phase pick information for each event;
2. and a single control file that details all available picks.

In [ ]:
# --- Imports ---
import pathlib

import numpy as np
import obspy
import pandas as pd

## Pick files

***File format***

The first line should contain the total number of picks, followed by rows containing:

| latitude | longitude | depth | traveltime | error |
| --- | --- | --- | --- | --- |
| ... | ... | ... | ... | ... |

for each pick.

## Control file
If you do not want to simultaneously invert for earthquake hypocentres as well as the velocity model, we can treat the receivers as pseudo-sources. This allows us to greatly speed up the inversion as we only need to compute the forward problem for each station (order of 10s), rather than for each earthquake (order of 1000s).

***File format***

The first line should contain the total number of sources, followed by a source line:

| latitude | longitude | depth |
| --- | --- | --- |

Then a line containing the total of picks for that source, followed by the phase and total phase count so far:

| phase | phase count |
| --- | --- |

In [ ]:
# --- Functions ---
def obspy2fmtomo(catalogue, stations, output_dir, phases):
    """
    Generate input files for the FMTOMO software package from an ObsPy
    catalogue.

    Parameters
    ----------
    catalogue : `obspy.Catalog` object
        Contains a list of `obspy.Event` objects, detailing origin times and
        phase picks.
    stations : `pandas.DataFrame` object
        DataFrame containing station information (latitude/longitude/elevation
        and a uid).
    output_dir : str
        Directory in which to save the output.
    phases : list of str
        List of phases to include in the output files.

    """

    output_dir = pathlib.Path(output_dir)
    output_dir.mkdir(exist_ok=True, parents=True)

    pick_cols = ["olat", "olon", "odep", "ttime", "ttime_err"]
    pick_dict = {}
    for i, station in stations.iterrows():
        stat = station["Name"]
        print(f"Creating pick file for {stat}...")

        # Create DataFrames to store all picks for each phase
        dfs = {}
        for phase in phases:
            dfs[phase] = pd.DataFrame(columns=pick_cols)

        for event in catalogue:
            origin = event.preferred_origin()
            olat, olon, odep = origin.latitude, origin.longitude, origin.depth
            otime = origin.time
            for pick in event.picks:
                if pick.waveform_id.station_code == stat:
                    line = pd.DataFrame([float(olat), float(olon), float(odep),
                                         float(pick.time - otime),
                                         float(pick.time_errors.uncertainty)],
                                        index=pick_cols).T
                    phase = pick.phase_hint
                    if phase in phases:
                        dfs[phase] = pd.concat([dfs[phase], line])

        for phase in phases:
            out = output_dir / "picks"
            out.mkdir(parents=True, exist_ok=True)
            outfile = out / f"{stat}.{phase}pick"
            out_df = dfs[phase]
            if out_df.empty:
                continue
            out_df = out_df.apply(pd.to_numeric)
            with outfile.open("w") as f:
                print(f"{len(out_df)}", file=f)
                for i, pick in out_df.iterrows():
                    print((f"{pick.olat:.4f} {pick.olon:.4f} "
                           f"{pick.odep/1000:.5f} {pick.ttime:.4f} "
                           f"{pick.ttime_err:.2f}"), file=f)

        anypicks = [dfs[phase].empty for pick in phases]
        if not np.all(anypicks):
            pick_dict[stat] = dfs
    print("Generation of pick files complete.")

    with (output_dir / "pick.control").open("w") as f:
        print(f"{len(pick_dict)}", file=f)
        for key, value in pick_dict.items():
            station = stations[stations["Name"] == key].iloc[0]
            stat = station["Name"]
            print((f"{station.Latitude:.4f} "
                   f"{station.Longitude:.4f} "
                   f"{station.Elevation:.4f}"), file=f)

            print(f"{len(phases)}", file=f)
            for phase in phases:
                print(f"1 1 {stat}.{phase}pick", file=f)

    stat_df = pd.DataFrame(columns=["Name", "Latitude", "Longitude", "Elevation"])
    for key in pick_dict.keys():
        stat_df = pd.concat([stat_df, stations[stations["Name"] == key]])

    stat_df.to_csv(str(output_dir / "stations_file.txt"), index=False)


## Creating inputs

The input files should be output to the `mkmodel` directory of your FMTomo run - set below

In [ ]:
mkmodel = "/path/to/mkmodel"

### Station files

You must provide a file containing an initial list of stations. A station file for use in FMTomo is produced from those for which there exist picks in the event catalogue.

In [ ]:
stations = pd.read_csv("/path/to/station_file")

# Elevation must be in km and positive UP - apply any necessary corrections
# Metres -> Kilometres
# stations["Elevation"] = stations["Elevation"].apply(lambda x: x / 1000)

# Positive-down -> Positive-up
# stations["Elevation"] = stations["Elevation"].apply(lambda x: x / -1)

### Reading in event catalogues

#### Read events from .hyp file

The .hyp file format extension is used for the outputs from NonLinLoc. ObsPy provide a parser that can read these files and build an ObsPy Catalog, which can then be parsed into the inputs for FMTomo

In [ ]:
cat = obspy.read_events("/path/to/.hyp")

In [ ]:
print(cat)

#### Read events from QuakeMigrate run

QuakeMigrate provides a utility function that will read the outputs of a given run into an ObsPy Catalog, which can then be parsed into the inputs for FMTomo

In [ ]:
import quakemigrate.export.to_obspy as obspy_catalog

cat = obspy_catalog.read_quakemigrate("/path/to/run")

In [ ]:
print(cat)

### Create inputs

Note: Using the same output path for P and S (if run separately) will overwrite the `pick.control` file. Either make a copy or choose a different output directory.

In [ ]:
# P phase picks
obspy2fmtomo(cat, stations, mkmodel, ["P"])

In [ ]:
# S phase picks
obspy2fmtomo(cat, stations, mkmodel, ["S"])